In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df= pd.read_csv(r'/content/drive/MyDrive/qa.csv')

In [5]:
df.drop('Unnamed: 2' , axis=1, inplace=True)

In [6]:
df.head(5)

,Questions,Answers
0,How to signup in the AI testing tool?,Click on the sign up button
1,How to sign up for AI testing tool,Click on the sign up button as shown in the image
2,Why I am getting an error while login?,Either the password or username is incorrect. ...
3,Why I am not able to login to AI testing tool?,Username and Password is incorrect. Please ent...
4,How to create a new project?,Go on to Dashboard and click on add project. Y...


In [7]:
!pip install transformers
from transformers import BertTokenizer
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 31.2 MB/s eta 0:00:00


In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
df.shape

(125, 2)

In [9]:
import pandas as pd
from transformers import pipeline

# load pre-trained BERT model
nlp = pipeline("text2text-generation", model="bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model 'BertForMaskedLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [10]:
!pip install nlpaug


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 32.7 MB/s eta 0:00:00


In [11]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.augmenter.audio as naa

In [12]:
df.isna().sum()

Questions    0
Answers      8
dtype: int64

In [13]:
df.dropna(inplace=True, axis=0)

In [14]:
df.isna().sum()

Questions    0
Answers      0
dtype: int64

In [15]:
df.shape

(117, 2)

In [16]:
import pandas as pd
import nlpaug.augmenter.word as naw



# Define data augmentation techniques
# Positive examples
aug_synonym = naw.SynonymAug()
aug_paraphrase = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action='substitute')
aug_insert = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

# Negative examples
aug_delete = naw.RandomWordAug(action='delete')
aug_swap = naw.RandomWordAug(action='swap')
aug_random_insert = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
aug_random_delete = naw.RandomWordAug(action='crop')

# Define data augmentation functions
def augment_positive_example(question, answer):
    augmented_pairs = []
    # Augment using synonym replacement
    """for i in range(10):
        augmented_question = aug_synonym.augment(question)
        augmented_answer = aug_synonym.augment(answer)
        augmented_pairs.append((augmented_question, augmented_answer, 1))

    # Augment using paraphrasing
    for i in range(3):
        augmented_question = aug_paraphrase.augment(question)
        augmented_answer = aug_paraphrase.augment(answer)
        augmented_pairs.append((augmented_question, augmented_answer, 1))"""

    # Augment using insertion
    for i in range(10):
        augmented_question = aug_insert.augment(question)
        augmented_answer = aug_insert.augment(answer)
        augmented_pairs.append((augmented_question, augmented_answer, 1))

    return augmented_pairs

def augment_negative_example(question, answer):
    augmented_pairs = []
    # Augment using deletion
    for i in range(3):
        augmented_question = aug_delete.augment(question)
        augmented_pairs.append((augmented_question, answer, 0))

    # Augment using random swap
    for i in range(3):
        augmented_question = aug_swap.augment(question)
        augmented_pairs.append((augmented_question, answer, 0))

    # Augment using random insertion
    for i in range(2):
       augmented_question = aug_random_insert.augment(question)
       augmented_pairs.append((augmented_question, answer, 0))

    # Augment using random deletion
    for i in range(2):
        augmented_question = aug_random_delete.augment(question)
        augmented_pairs.append((augmented_question, answer, 0))

    return augmented_pairs

# Apply data augmentation on each question-answer pair in the dataframe
all_examples = []
for i, row in df.iterrows():
    question = row['Questions']
    answer = row['Answers']
    positive_pairs = augment_positive_example(question, answer)
    negative_pairs = augment_negative_example(question, answer)
    all_examples += positive_pairs + negative_pairs

# Save augmented data to a new dataframe
aug_df = pd.DataFrame(all_examples, columns=['Question', 'Answer', 'Label'])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [17]:
aug_df.to_csv('aug_text_posneg_para.csv')